# Autoencoder-for-HTTP-Log-Anomaly-Detection

Syslog 데이터를 이용하여 Autoencoder Anomaly Detection 진행.

---

In [2]:
# 파일 확인
!ls "./data"
import os
os.chdir("./data")

HDFS_100k.log_anomaly_label.csv  log_model_HDFS-testing.h5  syslog.xls
HDFS_100k.log_structured.csv	 log_model-testing.h5	    weblog.csv


In [3]:
# import section
from __future__ import print_function
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras.models import  Model
from keras.layers import Dense, Input
import numpy as np
import pandas as pd

In [4]:
df = pd.read_excel('syslog.xls')

In [5]:
df

,심각도,발생 일시,그룹경로,시스템명,호스트명,리소스 이름,대상,이벤트 타입,이벤트 소스,내용
0,ERROR,2021-01-08 11:39:07,인프라그룹,Syslog Receiver,NaN,Syslog Receiver,인프라그룹>Syslog Receiver,syslog,172.30.151.88,[syslog]<43>Jan 8 11:39:07 Kolon_L7_trust_02 ...
1,ERROR,2021-01-08 11:39:07,인프라그룹,Syslog Receiver,NaN,Syslog Receiver,인프라그룹>Syslog Receiver,syslog,172.30.151.88,[syslog]<43>Jan 8 11:39:07 Kolon_L7_trust_02 ...
2,ERROR,2021-01-08 11:39:06,인프라그룹,Syslog Receiver,NaN,Syslog Receiver,인프라그룹>Syslog Receiver,syslog,172.30.151.87,[syslog]<43>Jan 8 11:39:06 Kolon_L7_trust_01 ...
3,ERROR,2021-01-08 11:38:07,인프라그룹,Syslog Receiver,NaN,Syslog Receiver,인프라그룹>Syslog Receiver,syslog,172.30.151.88,[syslog]<43>Jan 8 11:38:07 Kolon_L7_trust_02 ...
4,ERROR,2021-01-08 11:38:07,인프라그룹,Syslog Receiver,NaN,Syslog Receiver,인프라그룹>Syslog Receiver,syslog,172.30.151.88,[syslog]<43>Jan 8 11:38:07 Kolon_L7_trust_02 ...
...,...,...,...,...,...,...,...,...,...,...
9995,ERROR,2021-01-06 18:32:06,인프라그룹,Syslog Receiver,NaN,Syslog Receiver,인프라그룹>Syslog Receiver,syslog,172.30.151.88,[syslog]<43>Jan 6 18:32:06 Kolon_L7_trust_02 ...
9996,ERROR,2021-01-06 18:31:10,인프라그룹,Syslog Receiver,NaN,Syslog Receiver,인프라그룹>Syslog Receiver,syslog,172.30.151.87,[syslog]<43>Jan 6 18:31:10 Kolon_L7_trust_01 ...
9997,ERROR,2021-01-06 18:31:10,인프라그룹,Syslog Receiver,NaN,Syslog Receiver,인프라그룹>Syslog Receiver,syslog,172.30.151.87,[syslog]<43>Jan 6 18:31:10 Kolon_L7_trust_01 ...
9998,ERROR,2021-01-06 18:31:06,인프라그룹,Syslog Receiver,NaN,Syslog Receiver,인프라그룹>Syslog Receiver,syslog,172.30.151.88,[syslog]<43>Jan 6 18:31:06 Kolon_L7_trust_02 ...


In [6]:
df.isna().sum()

심각도           0
발생 일시         0
그룹경로          0
시스템명          0
호스트명      10000
리소스 이름        0
대상            0
이벤트 타입        0
이벤트 소스        0
내용            0
dtype: int64

In [7]:
for i in df.columns:
    print(i)
    print(df[i].factorize())
    print(' === ')

심각도
(array([0, 0, 0, ..., 0, 0, 0]), Index(['ERROR'], dtype='object'))
 === 
발생 일시
(array([   0,    0,    1, ..., 4408, 4409, 4409]), Index(['2021-01-08 11:39:07', '2021-01-08 11:39:06', '2021-01-08 11:38:07',
       '2021-01-08 11:38:06', '2021-01-08 11:37:07', '2021-01-08 11:37:06',
       '2021-01-08 11:36:07', '2021-01-08 11:36:06', '2021-01-08 11:35:07',
       '2021-01-08 11:35:06',
       ...
       '2021-01-06 18:35:10', '2021-01-06 18:35:06', '2021-01-06 18:34:10',
       '2021-01-06 18:34:06', '2021-01-06 18:33:10', '2021-01-06 18:33:06',
       '2021-01-06 18:32:10', '2021-01-06 18:32:06', '2021-01-06 18:31:10',
       '2021-01-06 18:31:06'],
      dtype='object', length=4410))
 === 
그룹경로
(array([0, 0, 0, ..., 0, 0, 0]), Index(['인프라그룹'], dtype='object'))
 === 
시스템명
(array([0, 0, 0, ..., 0, 0, 0]), Index(['Syslog Receiver'], dtype='object'))
 === 
호스트명
(array([-1, -1, -1, ..., -1, -1, -1]), Float64Index([], dtype='float64'))
 === 
리소스 이름
(array([0, 0, 0, ..., 0, 0, 0]), Index

In [20]:
df.내용.str.split(' ').str[0].factorize()

(array([0, 0, 0, ..., 0, 0, 0]),
 Index(['[syslog]<43>Jan', '[user]<11>1', '[local0]<131>Jan'], dtype='object'))